In [1]:
from enum import Enum


class StringEnum(str, Enum):
    @classmethod
    def allowed(cls) -> tuple[str, ...]:
        return tuple(e.value for e in cls)


class HookTypes(StringEnum):
    """
    Enum for hook types.
    """

    PRE = "pre"
    POST = "post"
    ON_ERROR = "on_error"

In [2]:
HookTypes.allowed()

('pre', 'post', 'on_error')

In [6]:
hook = HookTypes.PRE
hook2 = "pre"

f = lambda hook: (
    True if isinstance(hook, HookTypes) else hook in HookTypes.allowed()
)
print(f(hook))  # True
print(f(hook2))  # True

True
True


In [ ]:
from dataclasses import dataclass, field
from typing import ClassVar


@dataclass(slots=True, frozen=True, init=False)
class Params:
    """Base class for parameters used in various functions."""

    _strict: ClassVar[bool] = field(default=False, repr=False, init=False)
    _allowed_keys: ClassVar[set[str]] = field(
        default=set(), init=False, repr=False
    )

    @classmethod
    def allowed(cls) -> set[str]:
        """Return the keys of the parameters."""
        if cls._allowed_keys:
            return cls._allowed_keys
        cls._allowed_keys = {
            i for i in cls.__dataclass_fields__.keys() if not i.startswith("_")
        }
        return cls._allowed_keys

    def to_dict(self) -> dict[str, str]:
        data = {}
        for k in self.allowed():
            if (v := getattr(self, k)) is not Unset:
                data[k] = v

In [4]:
print(Params._allowed_keys)
Params.allowed()
print(Params._allowed_keys)

set()
{'mono'}


In [ ]:
import re
from lionagi._utils import Undefined, Unset, is_sentinel

from typing import Any, ClassVar


@dataclass(slots=True, frozen=True, init=False)
class Params:
    """Base class for parameters used in various functions."""

    _none_as_sentinel: ClassVar[bool] = field(
        default=False, repr=False, init=False
    )
    """If True, None is treated as a sentinel value."""

    _strict: ClassVar[bool] = field(default=False, repr=False, init=False)
    """No sentinels allowed if strict is True."""

    _allowed_keys: ClassVar[set[str]] = field(
        default=set(), init=False, repr=False
    )
    """Class variable cache to store allowed keys for parameters."""

    _prefill_unset: ClassVar[bool] = field(
        default=True, repr=False, init=False
    )

    @classmethod
    def _is_sentinel(cls, value: Any) -> bool:
        """Check if a value is a sentinel (Undefined or Unset)."""
        if value is None and cls._none_as_sentinel:
            return True
        return is_sentinel(value)

    def _validate_strict_unset(self) -> None:
        def _validate_strict(k):
            if self._strict and self._is_sentinel(getattr(self, k, Unset)):
                raise ValueError(f"Missing required parameter: {k}")
            if (
                self._prefill_unset
                and getattr(self, k, Undefined) is Undefined
            ):
                object.__setattr__(self, k, Unset)

        for k in self.allowed():
            _validate_strict(k)

    def __post_init__(self):
        """Post-initialization to ensure all fields are set."""
        self._validate_strict_unset()

    @classmethod
    def allowed(cls) -> set[str]:
        """Return the keys of the parameters."""
        if cls._allowed_keys:
            return cls._allowed_keys
        cls._allowed_keys = {
            i for i in cls.__dataclass_fields__.keys() if not i.startswith("_")
        }
        return cls._allowed_keys

    def to_dict(self) -> dict[str, str]:
        data = {}
        for k in self.allowed():
            if not self._is_sentinel(v := getattr(self, k)):
                data[k] = v
        return data